# Travel agency's review generator

Implement a text generator, which imitates reviewers of a travel agency using the LSTM language model.

In [ ]:
import pandas as pd

reviews = pd.read_csv('https://raw.githubusercontent.com/mlcollege/natural-language-processing/master/data/en_reviews.csv', sep='\t', header=None, names =['rating', 'text'])
reviews = reviews['text'].tolist()
print(reviews[:2])

['A voucher to nowhere #skypickerfail 2400 out of pocket due to skypicker delays in their booking office', 'I booked with Kiwi for the first time, just a short flight from Göteborg to London. I had forgotten my middle name in the fill-out section and was quite worried I had to pay for another ticket. Dominika and Nikola resolved the situation in good time with no extra cost. Thank you very much, will be booking again!']


## Prepare training data

Create one long string of all reviews separated by the new line symbol. First, we need to replace all new lines from the source data with spaces.

In [ ]:
data = '\n'.join(map(lambda x: x.replace('\n', ' '), reviews))

Create a list of all characters and their integer representations.

In [ ]:
chars = sorted(list(set(data)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(len(chars))

184


Create character sequences of MAXLEN length and a list that stores the characters occuring right after the sequence in the training data. The next sequence starts STEP characters after the beginning of the previous sequence.

In [ ]:
MAXLEN = 40
STEP = 10

sequences = []
next_chars = []
for i in range(0, len(data) - MAXLEN, STEP):
    sequences.append(data[i: i + MAXLEN])
    next_chars.append(data[i + MAXLEN])


Create the training data

In [ ]:
import numpy as np

X_train = np.zeros((len(sequences), MAXLEN, len(chars)), dtype=bool)
y_train = np.zeros((len(sequences), len(chars)), dtype=bool)

for i, sequences in enumerate(sequences):
    for t, char in enumerate(sequences):
        X_train[i, t, char_indices[char]] = 1
        y_train[i, char_indices[next_chars[i]]] = 1

Build an LSTM language model

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Input(shape=(MAXLEN, len(chars))))
model.add(LSTM(512, dropout=0.5, return_sequences=True))
model.add(LSTM(512, dropout=0.5))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Define a helper function for sampling characters from a probability distribution.

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Define the function which generates reviews using an LSTM language model.

In [ ]:
def generate(seed, temperature=1.0):
    sentence = MAXLEN * '\n' + seed
    sentence = sentence[-MAXLEN:]
    generated = seed

    next_char = None
    while next_char != '\n':
        X_pred = np.zeros((1, MAXLEN, len(chars)))
        for t, char in enumerate(sentence):
            X_pred[0, t, char_indices[char]] = 1.

        y_pred = model.predict(X_pred, verbose=0)[0]
        next_index = sample(y_pred, temperature)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated[0:-1]

Train the model and print a sample after every iteration

In [ ]:
EPOCHS = 5

old_loss = None
for iteration in range(1, EPOCHS + 1):
    print()
    print('-' * 50)
    print('Iteration', iteration)

    history = model.fit(X_train, y_train, batch_size=512, epochs=1)
    loss = history.history['loss'][0]
    if old_loss != None and old_loss < loss:
        print("Loss explosion.")
        break
    old_loss = loss
    start_index = np.random.randint(0, len(data) - MAXLEN - 1)
    sentence = data[start_index: start_index + MAXLEN]
    print(generate(sentence))

In [ ]:
#model.save_weights("lstm_lm.h5")

In [ ]:
!wget https://www.mlcollege.com/data/lstm_lm.h5
model.load_weights("lstm_lm.h5")

In [ ]:
generate('The service was')